In [54]:
import structure as st
from tkinter import filedialog
import customtkinter as ctk
from CTkMessagebox import CTkMessagebox
import re
import os
import ctypes
from ctypes import wintypes

In [105]:
def get_desktop_path():
    CSIDL_DESKTOP = 0x0000  # Desktop folder constant
    SHGFP_TYPE_CURRENT = 0
    
    buf = ctypes.create_unicode_buffer(wintypes.MAX_PATH)
    ctypes.windll.shell32.SHGetFolderPathW(None, CSIDL_DESKTOP, None, SHGFP_TYPE_CURRENT, buf)
    
    return buf.value
    
def change_to_c(cds=None,path=None):
    # if path!=None:
    #     cds=st.fasta_to_list(fasta_dir=path,seq_to_codon=True,sos_eos=False,remove_stop=False)[0]
    #     gc_initial = round((cds.count('C')+cds.count('G')+cds.count('c')+cds.count('g'))/len(cds.replace(' ',''))*100,1)
    # else:
    only_letters = re.sub(r'[^AGCTUagctu]', '', cds)
    cds = st.seq_to_cds([only_letters],sep=' ',sos_eos=False,remove_stop=False)[0] 
    gc_initial = round((only_letters.count('C')+only_letters.count('G')+only_letters.count('c')+only_letters.count('g'))/len(only_letters)*100,1)
    cds = cds.split(' ')
    exception_cds=['ATG','TTA','TTG','AAA','AAG','GAA','GAG','CAA','CAG','AGA','AGG','TGG','TAA','TAG','TGA','AUG','UUA','UUG','UGG','UAA','UAG','UGA']
    for i in range(len(cds)):
        if cds[i] not in exception_cds:
            cds[i]=cds[i][:-1]+'C'
    cds = ''.join(cds)
    if path is not None:
        filename = "CONVERTED_SEQ.txt"
        directory_path = os.path.dirname(path)
        path_saved = f"{directory_path}/{filename}"
    else:
        desktop = get_desktop_path()
        filename = "CONVERTED_SEQ.txt"
        path_saved = os.path.join(desktop,filename)
    with open(path_saved, 'w') as file:
        file.write(cds) 
    gc_converted = round((cds.count('C')+cds.count('G')+cds.count('c')+cds.count('g'))/len(cds)*100,1)
    return cds, gc_initial, gc_converted, path_saved
    
def open_file():
    file_path = filedialog.askopenfilename(filetypes=[("FASTA files", "*.fasta"), ("Text files", "*.txt")])
    if file_path:
        valid_characters=['A', 'G', 'C', 'T','U']
        cds=st.fasta_to_list(fasta_dir=file_path,seq_to_codon=False,sos_eos=False)[0]
        seq_entry.delete('1.0',ctk.END)
        if not set(cds.upper()) in valid_characters:
            cds=''.join([c for c in cds if c.upper() in valid_characters])
        seq_entry.insert('1.0',cds)
        result, gc_initial, gc_converted, path_saved = change_to_c(cds = cds, path = file_path)  
        gc_label_initial.configure(text=f'GC content: {gc_initial}%')  
        gc_label_converted.configure(text=f'GC content: {gc_converted}%')
        result_entry.configure(state=ctk.NORMAL)
        result_entry.delete("1.0", ctk.END)
        result_entry.insert('1.0',result) 
        result_entry.configure(state=ctk.DISABLED)
        save_label.configure(text=f'Converted Sequence Saved in:\n {path_saved}')
            
def submit_sequence():
    sequence = seq_entry.get("1.0", ctk.END)
    cleaned_text = sequence.strip().replace('\n','')
    valid_characters=['A', 'G', 'C', 'T','U']
    if not set(cleaned_text.upper()) in valid_characters:
        cds=''.join([c for c in cleaned_text if c.upper() in valid_characters])
        seq_entry.delete('1.0',ctk.END)
        seq_entry.insert('1.0',cds)
    result, gc_initial, gc_converted, path_saved = change_to_c(cds=cds)  
    gc_label_initial.configure(text=f'GC content: {gc_initial}%')  
    gc_label_converted.configure(text=f'GC content: {gc_converted}%')
    result_entry.configure(state=ctk.NORMAL)
    result_entry.delete("1.0", ctk.END)
    result_entry.insert('1.0',result.upper()) 
    result_entry.configure(state=ctk.DISABLED)
    save_label.configure(text=f'Converted Sequence Saved in:\n {path_saved}')
    codon_seperation()

def codon_seperation():
    if check_var.get()==1:
        entry_sequence = seq_entry.get("1.0", ctk.END)
        seq_entry.delete('1.0',ctk.END)
        only_letters = re.sub(r'[^A-Za-z]', '', entry_sequence)
        cds = st.seq_to_cds([only_letters],sep=' ',sos_eos=False,remove_stop=False)[0] 
        seq_entry.insert('1.0',cds)
        gc_initial = round((cds.count('C')+cds.count('G')+cds.count('c')+cds.count('g'))/len(cds.replace(' ',''))*100,1)  
        gc_label_initial.configure(text=f'GC content: {gc_initial}% (*calculated based on the codon numbers)')
        
        result_entry.configure(state=ctk.NORMAL)
        result_seq = result_entry.get("1.0", ctk.END)
        result_entry.delete('1.0',ctk.END)
        only_letters = re.sub(r'[^A-Za-z]', '', result_seq)
        cds = st.seq_to_cds([only_letters],sep=' ',sos_eos=False,remove_stop=False)[0] 
        result_entry.insert('1.0',cds)
        result_entry.configure(state=ctk.DISABLED)
        gc_converted = round((cds.count('C')+cds.count('G')+cds.count('c')+cds.count('g'))/len(cds.replace(' ',''))*100,1)  
        gc_label_converted.configure(text=f'GC content: {gc_converted}% (*calculated based on the codon numbers)')
    else:
        sequence = seq_entry.get("1.0", ctk.END)
        seq_entry.delete('1.0',ctk.END)
        only_letters = re.sub(r'[^A-Za-z]', '', sequence)
        cds = only_letters.replace(' ','')
        seq_entry.insert('1.0',cds)
        gc_initial = round((cds.count('C')+cds.count('G')+cds.count('c')+cds.count('g'))/len(cds.replace(' ',''))*100,1)  
        gc_label_initial.configure(text=f'GC content: {gc_initial}%')
        
        result_entry.configure(state=ctk.NORMAL)
        result_seq = result_entry.get("1.0", ctk.END)
        result_entry.delete('1.0',ctk.END)
        only_letters = re.sub(r'[^A-Za-z]', '', result_seq)
        cds = only_letters.replace(' ','')
        result_entry.insert('1.0',cds)
        result_entry.configure(state=ctk.DISABLED)
        gc_converted = round((cds.count('C')+cds.count('G')+cds.count('c')+cds.count('g'))/len(cds.replace(' ',''))*100,1)  
        gc_label_converted.configure(text=f'GC content: {gc_converted}%')

ctk.set_appearance_mode('dark')
ctk.set_default_color_theme('blue')

root = ctk.CTk()
root.title("Wobble to C converter")
root.geometry('600x600')

# root.rowconfigure((0,2,3,4,5,6,7),weight=1, uniform = 'a')
root.rowconfigure((1,4),weight=2, uniform = 'a')
root.rowconfigure((2,5),weight=6, uniform = 'a')
root.rowconfigure((0,3,6,7),weight=3, uniform = 'a')
root.columnconfigure((0,1,2),weight=1, uniform = 'a')

file_button = ctk.CTkButton(root, text="Select FASTA or TEXT File", command=open_file)
file_button.grid(row = 0, column = 0, padx = 5, pady = 5, sticky = 'w')

seq_label = ctk.CTkLabel(root, text="Or enter CDS sequence below",text_color='gray')
seq_label.grid(row = 1, column = 0, padx = 5, pady = 5, sticky = 'sw')

# Checkbox
check_var=ctk.IntVar()
check_box=ctk.CTkCheckBox(root,text='seperation based on codons',variable=check_var,command=codon_seperation)
check_box.grid(row = 1, column =1, padx = 5, pady = 5, sticky = 'sw')

# Input Textbox
seq_entry = ctk.CTkTextbox(root)
seq_entry.grid(row=2,column=0, sticky = 'nsew', padx = 5, pady = 5,columnspan=3)

#GC-content for initial codons
gc_label_initial=ctk.CTkLabel(root,text='')
gc_label_initial.grid(row=3,column=1, pady = 5, sticky = 'n')

# Submit button
submit_button = ctk.CTkButton(root, text="Submit", command=submit_sequence)
submit_button.grid(row = 3, column = 0, padx = 5, pady = 5, sticky = 'nw')

#GC-content for initial codons
Converted_seq_label=ctk.CTkLabel(root,text='Converted seq',text_color='gray')
Converted_seq_label.grid(row=4,column=0, pady = 5, sticky = 'sw')

# Result Textbox
result_entry = ctk.CTkTextbox(root)
result_entry.grid(row=5,column=0, sticky = 'nsew', padx = 5, pady = 5,columnspan=3)

# GC-content for converted codons
gc_label_converted=ctk.CTkLabel(root,text='')
gc_label_converted.grid(row=6,column=1, pady = 5, sticky = 'n')

# save label
save_label = ctk.CTkLabel(root,text='',text_color='gray')
save_label.grid(row=6,column=1,pady=10, sticky = 's')

# save label
developer_label = ctk.CTkLabel(root,text='barzegar@postech.ac.kr',text_color='gray')
developer_label.grid(row=7,column=1,pady=10, sticky = 's')

root.iconbitmap("C:\\Users\\farsh\\GitHub\\wobble_to_c\\gene-6.ico")
# Run the GUI
root.mainloop()

KeyboardInterrupt: 